In [10]:
import polars as pl
import json

# Load JSON mapping
column_mapping = {
  "age": ["Age", "age", "dob"],
  "gender": ["Gender", "sex", "Sex"],
  "race_ethnicity": ["Race", "Ethnicity", "race_ethnicity", "race", "race:AfricanAmerican", "race:Asian", "race:Caucasian", "race:Hispanic", "race:Other"],
  "region": ["Location", "region", "Region"],
  "health_issues": ["Conditions", "health_conditions", "Issues", "hypertension", "heart_disease"]
}

# Load your dataset
df = pl.read_csv('diabetes_dataset.csv')

# Function to rename columns based on mapping
def normalize_columns(df, mapping):
    for standard_name, possible_names in mapping.items():
        for col in possible_names:
            if col in df.columns:
                df = df.rename({col: standard_name})
                break
    return df

# Apply the normalization
df_normalized = normalize_columns(df, column_mapping)

# Display normalized DataFrame
df_normalized.head()

year,gender,age,location,race_ethnicity,race:Asian,race:Caucasian,race:Hispanic,race:Other,health_issues,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
i64,str,f64,str,i64,i64,i64,i64,i64,i64,i64,str,f64,f64,i64,i64
2020,"""Female""",32.0,"""Alabama""",0,0,0,0,1,0,0,"""never""",27.32,5.0,100,0
2015,"""Female""",29.0,"""Alabama""",0,1,0,0,0,0,0,"""never""",19.95,5.0,90,0
2015,"""Male""",18.0,"""Alabama""",0,0,0,0,1,0,0,"""never""",23.76,4.8,160,0
2015,"""Male""",41.0,"""Alabama""",0,0,1,0,0,0,0,"""never""",27.32,4.0,159,0
2016,"""Female""",52.0,"""Alabama""",1,0,0,0,0,0,0,"""never""",23.75,6.5,90,0


In [11]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
import numpy as np
import pandas as pd

class PatientAgent(Agent):
    def __init__(self, unique_id, model, age, gender, race, region, health_issues):
        super().__init__(unique_id, model)
        self.age = age
        self.gender = gender
        self.race = race
        self.region = region
        self.health_issues = health_issues
        self.consented = False

    def step(self):
        # Determine consent based on consent rate range
        consent_probability = random.uniform(self.model.consent_rate_min, self.model.consent_rate_max)
        self.consented = random.random() < consent_probability

class RecruitmentModel(Model):
    def __init__(self, df, consent_rate_min, consent_rate_max):
        self.df = df
        self.consent_rate_min = consent_rate_min
        self.consent_rate_max = consent_rate_max
        self.schedule = RandomActivation(self)

        # Create agents based on the DataFrame
        for i, row in df.iterrows():
            agent = PatientAgent(i, self, row['age'], row['gender'], row['race_ethnicity'], row['region'], row['health_issues'])
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()

# Function to run multiple simulations and calculate scores
def run_simulations(df, consent_rate_min, consent_rate_max, num_simulations):
    consent_results = []
    for _ in range(num_simulations):
        model = RecruitmentModel(df, consent_rate_min, consent_rate_max)
        for _ in range(1):  # Run for 1 step (as we only need to determine consent)
            model.step()
        consented_agents = sum([1 for agent in model.schedule.agents if agent.consented])
        consent_results.append(consented_agents)
    
    # Calculate statistics
    mean_consent = np.mean(consent_results)
    confidence_interval = np.std(consent_results) * 1.96 / np.sqrt(num_simulations)
    
    return {
        "mean_consent": mean_consent,
        "confidence_interval": confidence_interval,
        "consent_results": consent_results
    }

# Run the simulations
consent_rate_min = 0.4
consent_rate_max = 0.7
num_simulations = 100

# Convert Polars DataFrame to Pandas for compatibility with Mesa
df_normalized_pd = df_normalized.to_pandas()

simulation_results = run_simulations(df_normalized_pd, consent_rate_min, consent_rate_max, num_simulations)

# Display results
print(f"Mean Consent: {simulation_results['mean_consent']}")
print(f"Confidence Interval: +/- {simulation_results['confidence_interval']}")

KeyError: 'region'

In [13]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
%%writefile app.py

import streamlit as st
import pandas as pd
import polars as pl
import json
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
import numpy as np

# Load JSON mapping
column_mapping = {
  "age": ["Age", "age", "dob"],
  "gender": ["Gender", "sex", "Sex"],
  "race_ethnicity": ["Race", "Ethnicity", "race_ethnicity", "race", "race:AfricanAmerican", "race:Asian", "race:Caucasian", "race:Hispanic", "race:Other"],
  "region": ["Location", "region", "Region"],
  "health_issues": ["Conditions", "health_conditions", "Issues", "hypertension", "heart_disease"]
}

# Function to normalize columns
def normalize_columns(df, mapping):
    for standard_name, possible_names in mapping.items():
        for col in possible_names:
            if col in df.columns:
                df = df.rename({col: standard_name})
                break
    return df

class PatientAgent(Agent):
    def __init__(self, unique_id, model, age, gender, race, region, health_issues):
        super().__init__(unique_id, model)
        self.age = age
        self.gender = gender
        self.race = race
        self.region = region
        self.health_issues = health_issues
        self.consented = False

    def step(self):
        # Determine consent based on consent rate range
        consent_probability = random.uniform(self.model.consent_rate_min, self.model.consent_rate_max)
        self.consented = random.random() < consent_probability

class RecruitmentModel(Model):
    def __init__(self, df, consent_rate_min, consent_rate_max):
        self.df = df
        self.consent_rate_min = consent_rate_min
        self.consent_rate_max = consent_rate_max
        self.schedule = RandomActivation(self)

        # Create agents based on the DataFrame
        for i, row in df.iterrows():
            agent = PatientAgent(i, self, row['age'], row['gender'], row['race_ethnicity'], row['region'], row['health_issues'])
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()

# Streamlit App Configuration
st.set_page_config(page_title="Patient Recruitment Simulation", layout="wide")

st.title("Patient Recruitment Simulation")

# File uploader
data_file = st.file_uploader("Upload Data File", type=["csv", "tsv"])

if data_file is not None:
    df = pl.read_csv(data_file)
    df_normalized = normalize_columns(df, column_mapping)
    st.write("Data Preview:", df_normalized.head().to_pandas())

    # Inputs for simulation
    consent_rate_min = st.slider("Consent Rate Minimum", 0.0, 1.0, 0.4)
    consent_rate_max = st.slider("Consent Rate Maximum", 0.0, 1.0, 0.7)
    num_simulations = st.number_input("Number of Simulations", min_value=1, max_value=500, value=100)

    if st.button("Run Simulation"):
        # Convert Polars DataFrame to Pandas for compatibility with Mesa
        df_normalized_pd = df_normalized.to_pandas()
        
        # Run the simulations
        simulation_results = run_simulations(df_normalized_pd, consent_rate_min, consent_rate_max, num_simulations)
        
        # Display results
        st.write(f"Mean Consent: {simulation_results['mean_consent']}")
        st.write(f"Confidence Interval: +/- {simulation_results['confidence_interval']}")

Writing app.py


In [21]:

!npm install localtunnel


up to date, audited 23 packages in 418ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [22]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8000 & curl ipv4.icanhazip.com

& was unexpected at this time.


In [25]:
pip install ngrok


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
# Install necessary packages
!pip install pyngrok

# Save the Streamlit script to a file
with open("app.py", "w") as f:
    f.write('''<your streamlit code here>''')

# Run the Streamlit app and create a public URL using ngrok
from pyngrok import ngrok
import subprocess

# Run the Streamlit app
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Create a tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


t=2024-10-26T15:33:31-0400 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2024-10-26T15:33:31-0400 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2024-10-26T15:33:31-0400 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [31]:
streamlit run app.py

SyntaxError: invalid syntax (3737097518.py, line 1)